# BPE Tokenizer

Pada kode ini, ditunjukkan contoh penggunaan BPE Tokenizer buatan HuggingFace. [Petunjuk instalasi package](https://huggingface.co/docs/tokenizers/installation)

In [ ]:
# Import semua packages yang diperlukan.
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

In [ ]:
# Instantiate tokenizer BPE dengan unknown token [UNK] (berguna 
# saat testing jika menemukan misalnya karakter yang tidak dikenali).
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [ ]:
# Instantiate trainer BPE, PERHATIKAN hyperparameter vocab_size yang konsepnya
# mirip dengan `num_of_merges` yang diajarkan di kelas (relasinya kurang lebih 
# num_of_merges = vocab_size - num_of_alphabet).
trainer = BpeTrainer(
    vocab_size=10000,  # Perbesar/perkecil dan lihat bedanya pada hasil tokenisasi
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
)

In [ ]:
# Menambahkan pre-tokenizer yang berfungsi untuk memecah tiap kalimat/dokumen
# menjadi individual words berdasarkan keberadaan whitespace.
tokenizer.pre_tokenizer = Whitespace()

In [ ]:
# Mengekstrak kalimat-kalimat korpus ke dataset tersendiri sesuai spesifikasi
# input dari package tokenizer, yaitu file berisi raw text (tanpa hasil tokenisasi)
with open('train.txt', 'r') as train, open('train_text_only.txt', 'w') as out:
    lines = train.readlines()
    for line in lines:
        if line.startswith('# text ='):
            out.write(line[9:])

In [ ]:
# Melatih tokenizer dengan corpus.
tokenizer.train(
    files=["train_text_only.txt"],
    trainer=trainer
)

In [ ]:
# Contoh melakukan tokenisasi suatu kalimat
print(tokenizer.encode('Membaca Don Quixote untuk YouTube').tokens)
print(tokenizer.encode('Namun Rooney tidak sendirian.').tokens)

In [ ]:
with open('test.txt') as raw_text:
    corpus = raw_text.readlines()

In [ ]:
# Accumulating all sentences with their corresponding gold standard
sentences: list[str] = []
goldstd_sentences_tokens: list[list[str]] = []
index = 0

while index < 10415:
    sentences.append(corpus[index][9:])
    index += 1
    tok_res = []
    while not corpus[index].startswith('# text'):
        tok_res.append(corpus[index].strip())
        index += 1
        if index == 10415:
            break
    goldstd_sentences_tokens.append(tok_res)

In [ ]:
# Accumulating all sentences' tokenization results
tokenizer_sentences_tokens: list[list[str]] = []
for sentence in sentences:
    tokenizer_sentences_tokens.append(
        tokenizer.encode(sentence).tokens
    )

In [ ]:
# Counting accuracy
from collections import Counter
total_tokens = 0
correct_tokens = 0

for sentence_idx in range(len(tokenizer_sentences_tokens)):
    goldstd_sentence_tokens = goldstd_sentences_tokens[sentence_idx]
    tokenization_sentence_tokens = tokenizer_sentences_tokens[sentence_idx]
    total_tokens += len(goldstd_sentence_tokens)
    correct_tokens += len(list((Counter(goldstd_sentence_tokens) & Counter(tokenization_sentence_tokens)).elements()))

print("Accuracy of BPE: ", correct_tokens/total_tokens)